In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,CuDNNLSTM
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import scale

In [2]:
df=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

In [3]:
Y=df['SalePrice']
X=df.drop(['SalePrice','Id'],1)

In [4]:
X_new=pd.get_dummies(X)

In [5]:
X_filled=X_new.fillna(X_new.mean())

In [6]:
X_sc=scale(X_filled.iloc[:,:37])

C:\Users\User\Anaconda3\envs\inf552\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by the scale function.
  """Entry point for launching an IPython kernel.


In [7]:
X_sc.shape

(1460, 37)

In [8]:
column_name=X_filled.columns.tolist()[:37]

In [9]:
X_sc=pd.DataFrame(X_sc,columns=column_name)

In [10]:
X_sc=pd.concat([X_sc,X_filled.iloc[:,37:]],axis=1)

In [11]:
X_sc

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.073375,-2.293718e-01,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.511418,0.575425,-0.288653,...,0,0,0,1,0,0,0,0,1,0
1,-0.872563,4.519361e-01,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.574410,1.171992,-0.288653,...,0,0,0,1,0,0,0,0,1,0
2,0.073375,-9.311018e-02,0.073480,0.651479,-0.517200,0.984752,0.830215,0.323060,0.092907,-0.288653,...,0,0,0,1,0,0,0,0,1,0
3,0.309859,-4.564744e-01,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.574410,-0.499274,-0.288653,...,0,0,0,1,1,0,0,0,0,0
4,0.073375,6.336182e-01,0.375148,1.374795,-0.517200,0.951632,0.733308,1.364570,0.463568,-0.288653,...,0,0,0,1,0,0,0,0,1,0
5,-0.163109,6.790387e-01,0.360616,-0.795151,-0.517200,0.719786,0.491040,-0.574410,0.632450,-0.288653,...,0,0,0,1,0,0,0,0,1,0
6,-0.872563,2.248335e-01,-0.043379,1.374795,-0.517200,1.084115,0.975575,0.456019,2.029558,-0.288653,...,0,0,0,1,0,0,0,0,1,0
7,0.073375,6.454645e-16,-0.013513,0.651479,0.381743,0.057371,-0.574938,0.755176,0.910994,-0.090220,...,0,0,0,1,0,0,0,0,1,0
8,-0.163109,-8.652591e-01,-0.440659,0.651479,-0.517200,-1.333700,-1.689368,-0.574410,-0.973018,-0.288653,...,0,0,0,1,1,0,0,0,0,0
9,3.147673,-9.106796e-01,-0.310370,-0.795151,0.381743,-1.068734,-1.689368,-0.574410,0.893448,-0.288653,...,0,0,0,1,0,0,0,0,1,0


In [12]:
X_new.shape

(1460, 288)

In [119]:
model= Sequential()
model.add(Dense(256,input_dim=X_new.shape[1],activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16))
model.add(Dense(1))
model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])

In [120]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_65 (Dense)             (None, 256)               73984     
_________________________________________________________________
dense_66 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_67 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_68 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_69 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_70 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_71 (Dense)             (None, 1)                 17        
Total para

In [121]:
hist=model.fit(X_sc,Y,epochs=30,batch_size=10)

Epoch 1/30
1460/1460 [==============================] - 1s 863us/sample - loss: 20967630286.2466 - acc: 0.0000e+00
Epoch 2/30
1460/1460 [==============================] - 1s 591us/sample - loss: 1436883358.7945 - acc: 0.0000e+00
Epoch 3/30
1460/1460 [==============================] - 1s 471us/sample - loss: 1180296777.9178 - acc: 0.0000e+00
Epoch 4/30
1460/1460 [==============================] - 1s 497us/sample - loss: 1029313267.8904 - acc: 0.0000e+00
Epoch 5/30
1460/1460 [==============================] - 1s 629us/sample - loss: 974098108.6849 - acc: 0.0000e+00
Epoch 6/30
1460/1460 [==============================] - 1s 589us/sample - loss: 928608352.7397 - acc: 0.0000e+00
Epoch 7/30
1460/1460 [==============================] - 1s 574us/sample - loss: 894460627.2329 - acc: 0.0000e+00
Epoch 8/30
1460/1460 [==============================] - 1s 608us/sample - loss: 824971186.6301 - acc: 0.0000e+00
Epoch 9/30
1460/1460 [==============================] - 1s 599us/sample - loss: 802956733.0